# E T L

### 1. DESCOMPRESIÓN Y PREPARACIÓN DE LOS ARCHIVOS JSON

In [2]:
import gzip
import json
import pandas as pd
import re
import ast
from textblob import TextBlob
import requests

In [3]:
'''DESCOMPRIME, LEE EL ARCHIVO  Y CONVIERTE EL ARCHIVO steam_games.json.gz A UN ARCHIVO CSV'''
# Ruta al archivo comprimido
steam_games_json = r'G:\My Drive\HENRY\PROYECTO_INDIVIDUAL_I\PI01-ABRIL2024\DATASET\steam_games.json.gz'

# Lista para almacenar los DataFrames de cada objeto JSON
df_steam_games = []

# Descomprimir y leer el archivo JSON
with gzip.open(steam_games_json, 'rt', encoding='utf-8') as file:
    # Leer todo el contenido del archivo como una sola cadena
    content = file.read()
    # Dividir la cadena en líneas, cada línea es un objeto JSON
    json_objects = content.strip().split('\n')
    # Procesar y aplanar cada objeto JSON
    for json_str in json_objects:
        try:
            data = json.loads(json_str)
            df = pd.json_normalize(data)
            df_steam_games.append(df)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

# Concatenar todos los DataFrames
if df_steam_games:
    raw_games_df = pd.concat(df_steam_games, ignore_index=True)
    print(f"El DataFrame combinado tiene {len(raw_games_df)} filas.")
    print("Primeras filas del DataFrame combinado:")
    print(raw_games_df.head())
else:
    print("No se encontraron datos para cargar en DataFrames.")

# Guarda el dataframe en un archivo csv      
#raw_games_df.to_csv(r'G:\My Drive\HENRY\PROYECTO_INDIVIDUAL_I\PI01-ABRIL2024\DATASET\steam_games.json\output_steam_games.csv', index=False)

El DataFrame combinado tiene 120445 filas.
Primeras filas del DataFrame combinado:
  publisher genres app_name title  url release_date tags reviews_url specs  \
0       NaN    NaN      NaN   NaN  NaN          NaN  NaN         NaN   NaN   
1       NaN    NaN      NaN   NaN  NaN          NaN  NaN         NaN   NaN   
2       NaN    NaN      NaN   NaN  NaN          NaN  NaN         NaN   NaN   
3       NaN    NaN      NaN   NaN  NaN          NaN  NaN         NaN   NaN   
4       NaN    NaN      NaN   NaN  NaN          NaN  NaN         NaN   NaN   

  price early_access   id developer  
0   NaN          NaN  NaN       NaN  
1   NaN          NaN  NaN       NaN  
2   NaN          NaN  NaN       NaN  
3   NaN          NaN  NaN       NaN  
4   NaN          NaN  NaN       NaN  


In [11]:
'''DESCOMPRIME, LEE EL ARCHIVO  Y CONVIERTE EL ARCHIVO user_reviews.json.gz A UN ARCHIVO CSV'''

# Ruta al archivo comprimido
user_reviews_json = r'G:\My Drive\HENRY\PROYECTO_INDIVIDUAL_I\PI01-ABRIL2024\DATASET\user_reviews.json.gz'
df_user_reviews = []

def escape_quotes_in_reviews(line):
    # Esta función utiliza una expresión regular para escapar comillas dobles dentro de los valores del campo "review"
    # Asegura que solo se modifican las comillas dentro de los valores y no las que delimitan los campos JSON
    pattern = r'(?<=review": ")(.*?)(?="})'  # Captura el contenido dentro del campo "review"
    def replace(match):
        # Escapar todas las comillas dobles dentro del texto del review
        escaped_text = match.group(0).replace('"', '**')
        return escaped_text
    return re.sub(pattern, replace, line, flags=re.DOTALL)

with gzip.open(user_reviews_json, 'rt', encoding='utf-8') as file:
    line_num = 1
    for line in file:
        try:
            # Modifica cada línea para que obtenga el formato json necesario. Se hace un uso adecuado de comillas, escapes, booleanos, etc.
            line = line.replace("\"", '**')  
            line = line.replace("**}", '"}')  
            line = line.replace(': **', ': "')  
            line = line.replace("',", '",')  
            line = line.replace(", '", ', "')
            line = line.replace("':", '":') 
            line = line.replace(": '", ': "')  
            line = line.replace("'}", '"}')  
            line = line.replace("{'", '{"')  
            line = line.replace('\\', '')  
            line = line.replace("True", "true").replace("False", "false")
            corrected_line = escape_quotes_in_reviews(line)
            
            # Carga el json corregido a "data" si todo salió bien en la parte previa, de lo contrario avisa que hubo un error e imprime la línea problemática
            if corrected_line:
                #print(corrected_line)
                data = json.loads(corrected_line)
                df = pd.json_normalize(data)  # Normalizar y convertir a DataFrame
                df_user_reviews.append(df)
                #print(line)
            else:
                print("Error al procesar la línea:", line)
        
        # Si encuentra errores, imprime el error y la línea problemática.        
        except json.JSONDecodeError as e:
            print(f"Error en la línea {line_num}: {str(e)}")
            print("Error al procesar la línea:", line)
            
        line_num += 1

# Concatenar todos los DataFrames
if df_user_reviews:
    raw_reviews_df = pd.concat(df_user_reviews, ignore_index=True)
    print(f"El DataFrame combinado tiene {len(raw_reviews_df)} filas.")
    print("Primeras filas del DataFrame combinado:")
    print(raw_reviews_df.head())
else:
    print("No se encontraron datos para cargar en DataFrames.")

# Guarda el dataframe en un archivo csv      
#raw_reviews_df.to_csv(r'G:\My Drive\HENRY\PROYECTO_INDIVIDUAL_I\PI01-ABRIL2024\DATASET\user_reviews.json\australian_user_reviews.csv', index=False)

    


El DataFrame combinado tiene 25799 filas.
Primeras filas del DataFrame combinado:
             user_id                                           user_url  \
0  76561197970982479  http://steamcommunity.com/profiles/76561197970...   
1            js41637               http://steamcommunity.com/id/js41637   
2          evcentric             http://steamcommunity.com/id/evcentric   
3              doctr                 http://steamcommunity.com/id/doctr   
4          maplemage             http://steamcommunity.com/id/maplemage   

                                             reviews  
0  [{'funny': '', 'posted': 'Posted November 5, 2...  
1  [{'funny': '', 'posted': 'Posted June 24, 2014...  
2  [{'funny': '', 'posted': 'Posted February 3.',...  
3  [{'funny': '', 'posted': 'Posted October 14, 2...  
4  [{'funny': '3 people found this review funny',...  


In [5]:

'''DESCOMPRIME, LEE EL ARCHIVO  Y CONVIERTE EL ARCHIVO users_items.json.gz A UN ARCHIVO CSV - ESTE PROCESO TARDA APROX. 17 MINUTOS, NO ES EFICIENTE PERO FUNCIONA'''

user_items_json = r'G:\My Drive\HENRY\PROYECTO_INDIVIDUAL_I\PI01-ABRIL2024\DATASET\users_items.json.gz'


df_user_items = []

def escape_quotes_in_reviews(line):
    # Esta función utiliza una expresión regular para escapar comillas dobles dentro de los valores del campo "review"
    # Asegura que solo se modifican las comillas dentro de los valores y no las que delimitan los campos JSON
    pattern = r'(?<=item_name": ")(.*?)(?="})'  # Captura el contenido dentro del campo "review"
    def replace(match):
        # Escapar todas las comillas dobles dentro del texto del review
        escaped_text = match.group(0).replace('"', '**')
        return escaped_text
    return re.sub(pattern, replace, line, flags=re.DOTALL)


with gzip.open(user_items_json, 'rt', encoding='utf-8') as file:
    line_num = 1
    for line in file:
        try:
            # Modifica cada línea para que obtenga el formato json necesario. Se hace un uso adecuado de comillas, escapes, booleanos, etc.
            line = line.replace("\"", '**')  
            line = line.replace("**}", '"}')  
            line = line.replace(': **', ': "')
            line = line.replace('**,', '",')  
            line = line.replace("',", '",')  
            line = line.replace(", '", ', "')
            line = line.replace("':", '":') 
            line = line.replace(": '", ': "')  
            line = line.replace("'}", '"}')  
            line = line.replace("{'", '{"')  
            line = line.replace('\\', '')  
            line = line.replace("True", "true").replace("False", "false")
            corrected_line = escape_quotes_in_reviews(line)
            
            # Carga el json corregido a "data" si todo salió bien en la parte previa, de lo contrario avisa que hubo un error e imprime la línea problemática
            if corrected_line:
                data = json.loads(corrected_line)
                df = pd.json_normalize(data)  # Normalizar y convertir a DataFrame
                df_user_items.append(df)
           
            else:
                print("Error al procesar la línea:", line)
        
        # Si encuentra errores, imprime el error y la línea problemática.        
        except json.JSONDecodeError as e:
            print(f"Error en la línea {line_num}: {str(e)}")
            print("Error al procesar la línea:", line)
            
        line_num += 1

# Concatenar todos los DataFrames
if df_user_items:
    raw_uitems_df = pd.concat(df_user_items, ignore_index=True)
    print(f"El DataFrame combinado tiene {len(raw_uitems_df)} filas.")
    print("Primeras filas del DataFrame combinado:")
    print(raw_uitems_df.head())
else:
    print("No se encontraron datos para cargar en DataFrames.")
  
# Guarda el dataframe en un archivo csv  
#raw_uitems_df.to_csv(r'G:\My Drive\HENRY\PROYECTO_INDIVIDUAL_I\PI01-ABRIL2024\DATASET\users_items.json\australian_users_items.csv', index=False)


El DataFrame combinado tiene 88310 filas.
Primeras filas del DataFrame combinado:
             user_id  items_count           steam_id  \
0  76561197970982479          277  76561197970982479   
1            js41637          888  76561198035864385   
2          evcentric          137  76561198007712555   
3         Riot-Punch          328  76561197963445855   
4              doctr          541  76561198002099482   

                                            user_url  \
0  http://steamcommunity.com/profiles/76561197970...   
1               http://steamcommunity.com/id/js41637   
2             http://steamcommunity.com/id/evcentric   
3            http://steamcommunity.com/id/Riot-Punch   
4                 http://steamcommunity.com/id/doctr   

                                               items  
0  [{'item_id': '10', 'item_name': 'Counter-Strik...  
1  [{'item_id': '10', 'item_name': 'Counter-Strik...  
2  [{'item_id': '1200', 'item_name': 'Red Orchest...  
3  [{'item_id': '10', 'i

### 2. EXTRACCIÓN DE LA INFORMACIÓN
Lee los archivos csv's generados y crea nuevos que contienen la información estrictamente necesaria para alimentar las funciones que conformarán la API del proyecto.

#### 2.1 Crea el PARQUET que servirá para la función 1: PlaytimeGenre
NOTA: La base de datos que se genera a partir de los datos orginales posee un total de 9,993,947 de filas. Por lo tanto, se tomó la decisión de crear una muestra aleatoria con el 1% de los datos con el fin de prevenir problemas relacionados con el uso de memoria. De esta  manera  se logra probar  la correcta ejecución de funciones y deploy. 

In [4]:
###### 0. Lee los csv's de cada base de datos #####
raw_games_csv = pd.read_csv(r'G:\My Drive\HENRY\PROYECTO_INDIVIDUAL_I\PI01-ABRIL2024\DATASET\steam_games.json\output_steam_games.csv', dtype={0: str, 1: str, 2: str, 3: str, 4: str, 5: str, 6: str, 7: str, 8: str, 9: str, 10: str, 11:object, 12: str})
raw_uitems_csv = pd.read_csv(r'G:\My Drive\HENRY\PROYECTO_INDIVIDUAL_I\PI01-ABRIL2024\DATASET\users_items.json\australian_users_items.csv')

###### 1. Prepara el csv de raw_uitems_csv ######
# Configurar el campo 'items' para que sea interpretado correctamente y lea las listas
raw_uitems_csv['items'] = raw_uitems_csv['items'].apply(ast.literal_eval) # Realiza la conversión
#print(raw_uitems_csv['items'].apply(type).value_counts())  # Imprime el tipo de dato contenido en el campo 'items' y el número de filas

# Explota el dataframe para que se escriba un renglón por lista de cada usuario (se repite el usuario en cada fila)
uitems_exploded = raw_uitems_csv.explode('items')
# print(len(uitems_exploded)) # Imprime el nuevo número de filas (debe ser mucho mayor al del csv)

# Extraer claves de diccionario en nuevas columnas, se excluyen los valores NaN 
uitems_exploded['item_id'] = uitems_exploded['items'].apply(lambda x: x.get('item_id') if pd.notna(x) else x)
uitems_exploded['playtime_forever'] = uitems_exploded['items'].apply(lambda x: x.get('playtime_forever') if pd.notna(x) else x)

# Borra todas filas que contienen Nan en la columna de tiempo de juego (playtime_forever)
uitems_exploded= uitems_exploded.dropna(subset=['playtime_forever'])
# print (uitems_exploded['item_id'].isna().sum()) # R = 16,806 valores nulos
# print (uitems_exploded['playtime_forever'].isna().sum()) # R = 16,806 valores nulos

##### 2. Prepara el csv de raw_gamses_csv #####
# Borra todas filas que contienen solo Nan en todas sus columnas
raw_games_csv_clean = raw_games_csv.dropna(how='all')
raw_games_csv_clean = raw_games_csv_clean.dropna(subset=['genres'])
raw_games_csv_clean = raw_games_csv_clean.dropna(subset=['id'])

##### 3. JOIN: user items con steam games (Dataframe para responder función 1) #####
# Hacer un join  con las df raw_utiems y raw_games, pero solo conserva las columnas estrictamente necesarias
horas_jugadas = pd.merge(uitems_exploded[['item_id', 'playtime_forever']], raw_games_csv_clean[['id', 'release_date', 'genres']], left_on='item_id', right_on='id', how='inner')

# Convertir cadenas a  listas
horas_jugadas['genres'] = horas_jugadas['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Convertir 'release_date' a formato de fecha (datetime)
horas_jugadas['release_date'] = pd.to_datetime(horas_jugadas['release_date'],errors='coerce')

# Crea una nueva columna que solo contiene el año
horas_jugadas['release_year'] = horas_jugadas['release_date'].dt.year
# Conocer el periodo que abarca esta base de datos:
# print('Año minimo: ', horas_jugadas['release_year'].min())
# print('Año máximo: ', horas_jugadas['release_year'].max())

# Cada videojuego puede tener varias categorías  de  género, por lo cual, se expande este campo para contabilizar 
# por separado cada género. Se deja fuera la etiqueta de Early Access pues no se considera un género.
horas_jugadas = horas_jugadas.explode('genres')

# Genera una muestra aleatoria con el 
horas_jugadas = horas_jugadas.sample(frac = 0.01, random_state=42)
###------Crea el CSV que servirá para la función 1: PlaytimeGenre-----
#horas_jugadas.to_csv(r'G:\My Drive\HENRY\PROYECTO_INDIVIDUAL_I\PI01-ABRIL2024\DATASET\F01_PlaytimeGenre.csv', index=False)

###------Crea el parquet que servirá para la función 1: PlaytimeGenre-----
horas_jugadas.to_parquet(r'G:\My Drive\HENRY\PROYECTO_INDIVIDUAL_I\PI01-ABRIL2024\DATASET\F01_PlaytimeGenre.parquet', index=False)


#### 2.2 Crea el PARQUET que servirá para la función 2: UserForGenre
 NOTA: La base de datos que se genera a partir de los datos orginales posee un total de 9,993,947 de filas. Por lo tanto, se tomó la decisión de crear una muestra aleatoria con el 1% de los datos con el fin de prevenir problemas relacionados con el uso de memoria. De esta  manera  se logra probar  la correcta ejecución de funciones y deploy. 


In [6]:
##### 4. JOIN: user items con steam games (Dataframe para responder función 2) #####
# Hacer un join  con las df raw_utiems y raw_games, pero solo conserva las columnas estrictamente necesarias
horas_jugadas_usuario = pd.merge(uitems_exploded[['user_id','item_id', 'playtime_forever']], raw_games_csv_clean[['id', 'release_date', 'genres']], left_on='item_id', right_on='id', how='inner')

# Borra todas filas que contienen Nan en la columna de géneros (genres)
horas_jugadas_usuario = horas_jugadas_usuario.dropna(subset=['genres'])

# Borra todas filas que contienen Nan en la columna de user_id 
horas_jugadas_usuario = horas_jugadas_usuario.dropna(subset=['user_id'])

# Convertir cadenas a  listas
horas_jugadas_usuario['genres'] = horas_jugadas_usuario['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Convertir 'release_date' a formato de fecha (datetime)
horas_jugadas_usuario['release_date'] = pd.to_datetime(horas_jugadas_usuario['release_date'],errors='coerce')

# Crea una nueva columna que solo contiene el año
horas_jugadas_usuario['release_year'] = horas_jugadas_usuario['release_date'].dt.year

# Cada videojuego puede tener varias categorías  de  género, por lo cual, se expande este campo para contabilizar 
# por separado cada género. Se deja fuera la etiqueta de Early Access pues no se considera un género.
horas_jugadas_usuario = horas_jugadas_usuario.explode('genres')

horas_jugadas_usuario = horas_jugadas_usuario.sample(frac = 0.01, random_state=42)

###-----Crea el CSV que servirá para la función 2: UserForGenre----
#horas_jugadas_usuario.to_csv(r'G:\My Drive\HENRY\PROYECTO_INDIVIDUAL_I\PI01-ABRIL2024\DATASET\F02_UserForGenre.csv', index=False)

###-----Crea el parquet que servirá para la función 2: UserForGenre----
horas_jugadas_usuario.to_parquet(r'G:\My Drive\HENRY\PROYECTO_INDIVIDUAL_I\PI01-ABRIL2024\DATASET\F02_UserForGenre.parquet', index=False)


#### 2.3 Crea el PARQUET que servirá para la función 3: UsersRecommend

In [224]:
raw_reviews_csv = pd.read_csv(r'G:\My Drive\HENRY\PROYECTO_INDIVIDUAL_I\PI01-ABRIL2024\DATASET\user_reviews.json\australian_user_reviews.csv')

###### 1. Prepara el csv de raw_reviews_csv ######
# Configurar el campo 'reviews' para que sea interpretado correctamente y lea las listas
raw_reviews_csv['reviews'] = raw_reviews_csv['reviews'].apply(ast.literal_eval) # Realiza la conversión
#print(raw_uitems_csv['items'].apply(type).value_counts())  # Imprime el tipo de dato contenido en el campo 'items' y el número de filas

# Explota el dataframe para que se escriba un renglón por lista de cada usuario (se repite el usuario en cada fila)
reviews_exploded = raw_reviews_csv.explode('reviews')
# print(len(uitems_exploded)) # Imprime el nuevo número de filas (debe ser mucho mayor al del csv)

# Extraer claves de diccionario en nuevas columnas, se excluyen los valores NaN 
reviews_exploded['review'] = reviews_exploded['reviews'].apply(lambda x: x.get('review') if pd.notna(x) else x)
reviews_exploded['item_id'] = reviews_exploded['reviews'].apply(lambda x: x.get('item_id') if pd.notna(x) else x)
reviews_exploded['recommend'] = reviews_exploded['reviews'].apply(lambda x: x.get('recommend') if pd.notna(x) else x)

# Genera una función para hacer el análisis de sentimiento y crear una nueva columna con el resultado del análisis
def analyze_sentiment(df):
    """
    Analiza el sentimiento de las reseñas en un DataFrame y agrega una nueva columna con el resultado.
    
    Parámetros:
    - df (pandas.DataFrame): DataFrame que contiene las columnas de reseñas.
    
    Devuelve:
    - pandas.DataFrame: DataFrame con la columna de análisis de sentimiento agregada.
    """
    def get_sentiment(text):
        if pd.isna(text):
            return 1  # Neutral si la reseña está ausente
        analysis = TextBlob(text)
        if analysis.sentiment.polarity > 0.1:  # Umbral para positivo
            return 2  # Positivo
        elif analysis.sentiment.polarity < -0.1:  # Umbral para negativo
            return 0  # Negativo
        else:
            return 1  # Neutral

    # Aplicar la función de análisis de sentimiento a la columna de reseñas
    df['sentiment_analysis'] = df['review'].apply(get_sentiment)
 
    return df

# Aplicar la función
sentiment_function = analyze_sentiment(reviews_exploded)
sentiment_function = sentiment_function[['user_id','item_id','sentiment_analysis','recommend']]

# JOIN con el DF steam_games
# Hacer un join  con las df raw_utiems y raw_games, pero solo conserva las columnas estrictamente necesarias
game_reviews = pd.merge(sentiment_function[['user_id','item_id','sentiment_analysis','recommend']], raw_games_csv_clean[['id', 'release_date', 'title', 'developer']], left_on='item_id', right_on='id', how='inner')
# Convertir 'release_date' a formato de fecha (datetime)
game_reviews['release_date'] = pd.to_datetime(game_reviews['release_date'],errors='coerce')
# Crea una nueva columna que solo contiene el año
game_reviews['release_year'] = game_reviews['release_date'].dt.year

###-----Crea el CSV que servirá para la función 3: UsersRecommend----
#game_reviews.to_csv(r'G:\My Drive\HENRY\PROYECTO_INDIVIDUAL_I\PI01-ABRIL2024\DATASET\F03_UsersRecommend.csv', index=False)

###-----Crea el parquet que servirá para la función 3: UsersRecommend----
game_reviews.to_parquet(r'G:\My Drive\HENRY\PROYECTO_INDIVIDUAL_I\PI01-ABRIL2024\DATASET\F03_UsersRecommend.parquet', index=False)
#print(game_reviews)